In [1]:
import os

import numpy as np
from tensorflow.keras.models import *

from src.dlla.berg import make_mlp
from src.dlla.hw import prepare_traces_dl, dlla_known_p
from src.dlla.wegener import make_mlp_wegener, wegener_p
from src.tools.la import shuffle
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW

In [2]:
from src.tvla.cri import rho_test, tvla_t_test


def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [3]:
LIMITS = {
    Database.ascad: (None, 1000),
    Database.ascad_none: (None, 1000),
    Database.aisy: (1000, 1000),
}

PARAMS = {
    Database.ascad: {
        PollutionType.desync: np.arange(0, 2.05, .05),
        PollutionType.gauss: np.arange(0, 5.1, .1)
    },
    Database.ascad_none: {
        PollutionType.desync: np.arange(0, 205, 5),
        PollutionType.gauss: np.arange(0, 102, 2)
    },
    Database.aisy: {
        PollutionType.desync: np.arange(0, 460, 10),
        PollutionType.gauss: np.arange(0, 4100, 100)
    }
}

def get_params(db: Database, pollution_type: PollutionType):
    return PARAMS[db][pollution_type]

def get_limit(db: Database):
    return LIMITS[db]

def get_order(db: Database):
    if db is Database.ascad:
        return 2

    return 3

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        prof_lim, att_lim = get_limit(db)
        trace_set = TraceSetHW(db, pollution, (prof_lim, att_lim))

        if os.path.exists(trace_set.path):
            # print("Prepare traces (1/3)", end="\r")
            # x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            # print("Prepare traces (3/3)", end="\r")
            # x2b, y2b, x2b_att, y2b_att = prepare_traces_dl(*trace_set.profile_la(), *trace_set.attack_la(True), num_classes=2)
            #
            # print("Make model (1/3)    ", end="\r")
            # mdl_open = make_mlp(x9, y9, progress=False)
            # print("Make model (3/3)    ", end="\r")
            # mdl_wegener = make_mlp_wegener(x2b, y2b, False)

            tvla_x, tvla_y = shuffle(*trace_set.profile_la())
            print("TVLA                ", end="\r")
            # tvla_ps = np.min(tvla_t_test(tvla_x[:att_lim], tvla_y[:att_lim], get_order(db), progress=False), axis=1)
            rho_p = np.min(rho_test(tvla_x[:att_lim], tvla_y[:att_lim], progress=False), axis=1)[0]

            # print("Predict             ", end="\r")
            # dlla_k_p = dlla_known_p(mdl_open, x9_att, y9_att)
            # dlla_wegener_p = wegener_p(mdl_wegener, x2b_att, y2b_att)

            title = f"{pollution_type} ({param})."
            print(f"{title} TVLA t ({tvla_ps}). TVLA rho ({rho_p}). DLLA known key ({dlla_k_p}). DLLA Wegener ({dlla_wegener_p})")

            # for order, p in enumerate(tvla_ps):
            #     if order > 0:
            #         store_results(db, f"tvla_t_{order}", pollution, p)

            store_results(db, "tvla_rho", pollution, rho_p)
            # store_results(db, "dlla_open", pollution, dlla_k_p)
            # store_results(db, "dlla_wegener", pollution, dlla_wegener_p)

        print()

la_benchmark(Database.ascad_none, PollutionType.desync, [0])

PollutionType.desync (0). TVLA t ([1.00000000e+000 7.65394661e-117 1.66788366e-002 8.34506050e-002]). TVLA rho (0.0). DLLA known key (2.2131775667223047e-259). DLLA Wegener (5.351097043477547e-197)



In [4]:
def la_benchmark_all(db: Database):
    for poll in [PollutionType.desync, PollutionType.gauss]:
        la_benchmark(db, poll, get_params(db, poll))

while True:
    la_benchmark_all(Database.aisy)
    la_benchmark_all(Database.ascad)
    la_benchmark_all(Database.ascad_none)

PollutionType.desync (0). TVLA t ([1.00000000e+00 2.15683776e-13 2.14492322e-03 1.89580903e-01]). TVLA rho (4.133848818810293e-11). DLLA known key (3.7400148311999875e-134). DLLA Wegener (9.612118509297677e-33)

PollutionType.desync (10). TVLA t ([1.00000000e+00 2.26959307e-12 9.02376579e-04 2.13263593e-01]). TVLA rho (1.8046009131467144e-11). DLLA known key (1.030408901225921e-120). DLLA Wegener (1.0791074548134475e-30)

PollutionType.desync (20). TVLA t ([1.00000000e+00 5.40067355e-13 1.51530193e-03 1.58649647e-01]). TVLA rho (3.696820627396846e-12). DLLA known key (2.342749865259723e-135). DLLA Wegener (8.541950909495185e-34)

PollutionType.desync (30). TVLA t ([1.00000000e+00 1.83138867e-12 7.80079348e-04 1.70765108e-01]). TVLA rho (8.196738843224693e-10). DLLA known key (7.394400671244484e-116). DLLA Wegener (6.922422371645256e-46)

PollutionType.desync (40). TVLA t ([1.00000000e+00 1.18081281e-11 4.60385985e-04 1.47533550e-01]). TVLA rho (3.6785037238473706e-09). DLLA known key (

KeyboardInterrupt: 